In [ ]:

# ✅ LSTM 성능 리포트 (파일 경로 안전체크 포함)

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import os

# 1️⃣ 경로 세팅
csv_path = '../data/목적별_국적별_결측치.csv'
model_path = '../model/foreign_visitors_lstm_model.h5'

# 2️⃣ 파일 존재 여부 체크
assert os.path.exists(csv_path), f"CSV 파일이 존재하지 않습니다: {csv_path}"
assert os.path.exists(model_path), f"모델 파일이 존재하지 않습니다: {model_path}"

# 3️⃣ 데이터 로드
df = pd.read_csv(csv_path)
df = df.fillna(method='ffill')
df = df.sort_values(['country_code', 'purpose_code', 'year', 'month'])

features = ['visitors_num', 'lag_1', 'rolling_mean_3', 'rolling_mean_6',
            'rolling_mean_12', 'is_peak', 'is_holiday']

scaler = MinMaxScaler()
scaled = scaler.fit_transform(df[features])

# 4️⃣ Sliding Window
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i+n_steps])
        y.append(data[i+n_steps, 0])  # visitors_num
    return np.array(X), np.array(y)

n_steps = 12
X, y = create_sequences(scaled, n_steps)

# 5️⃣ 모델 로드
model = load_model(model_path)

# 6️⃣ 예측
y_pred_scaled = model.predict(X)

# 7️⃣ 복원
y_pred_full = np.hstack([y_pred_scaled, np.zeros((len(y_pred_scaled), len(features)-1))])
y_pred_inv = scaler.inverse_transform(y_pred_full)[:,0]

y_true_full = np.hstack([y.reshape(-1,1), np.zeros((len(y), len(features)-1))])
y_true_inv = scaler.inverse_transform(y_true_full)[:,0]

# 8️⃣ 성능 지표
mae = mean_absolute_error(y_true_inv, y_pred_inv)
rmse = mean_squared_error(y_true_inv, y_pred_inv) ** 0.5
mape = np.mean(np.abs((y_true_inv - y_pred_inv) / y_true_inv)) * 100

print(f'MAE: {mae:.2f}')
print(f'RMSE: {rmse:.2f}')
print(f'MAPE: {mape:.2f}%')

# 9️⃣ 예측 vs 실제 그래프
plt.figure(figsize=(12,6))
plt.plot(y_true_inv[:100], label='실제 입국자수')
plt.plot(y_pred_inv[:100], label='예측 입국자수', linestyle='--')
plt.title('예측 vs 실제 (샘플 100개)')
plt.xlabel('샘플')
plt.ylabel('입국자수')
plt.legend()
plt.show()
